In [1]:
import csv
import numpy as np
import pandas as pd
import time
from urllib2 import urlopen # python2
import urllib2
from itertools import product

In [2]:
di = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/MBiRW-master/Datasets/CDatasets/DiseasesName", header=None)
dr = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/MBiRW-master/Datasets/CDatasets/DrugsName", header=None)
didr = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/MBiRW-master/Datasets/CDatasets/DiDrAMat", header=None, sep='   ')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [3]:
print(len(di))
di = di.rename(columns={0:'disease_id'})
di.head(1)

409


,disease_id
0,D102100


In [4]:
print(len(dr))
dr = dr.rename(columns={0:'drug_id'})
dr.head(1)

663


,drug_id
0,DB00014


In [5]:
drug_id = dr['drug_id'].tolist()
didr.columns = drug_id
disease_id = di['disease_id'].tolist()
didr.index = disease_id
didr.head(1)

,DB00014,DB00035,DB00091,DB00104,DB00115,DB00122,DB00125,DB00126,DB00131,DB00136,...,DB08802,DB08803,DB08804,DB08820,DB08824,DB08835,DB08896,DB08901,DB08906,DB08907
D102100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
didr[drug_id[0]].ix[disease_id[disease_id.index('D114480')]]

1.0

In [7]:
pair = pd.DataFrame()
for i in disease_id:
    for j in drug_id : 
        if didr[j].ix[i] == 1:
            temp_df = pd.DataFrame([[i, j]], columns = ['disease_id', 'drug_id'])
            pair = pair.append(temp_df)
            

In [ ]:
pair.to_csv("/home/share/aschoi/data/Drug_Repositioning/7_new_association/bi_random_walk_Cdataset_association.tsv", sep='\t', index=False)

In [8]:
print len(pair)
pair.head(1)

2532


,disease_id,drug_id
0,D102100,DB00555


In [9]:
common_feature = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/5_common_feature.tsv", header = None)
drugs_with_common_feature = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/6_drugs_with_common_feature.tsv", header =None)

In [10]:
common_feature = common_feature.rename(columns = {0:'umls_id'})
drugs_with_common_feature = drugs_with_common_feature.rename(columns = {0:'DrugBank_ID'})

In [11]:
filterd_drug = dr[dr['drug_id'].isin(drugs_with_common_feature['DrugBank_ID'])]
print len(filterd_drug)
filterd_drug.head(1)

500


,drug_id
6,DB00125


In [12]:
filtered_asso = pair[pair['drug_id'].isin(filterd_drug['drug_id'])]
print len(filtered_asso)
filtered_asso.head()

1947


,disease_id,drug_id
0,D102100,DB00555
0,D102100,DB00949
0,D102100,DB01068
0,D102300,DB00190
0,D102300,DB00193


In [13]:
len(filtered_asso.drop_duplicates(subset='disease_id'))

364

In [14]:
filtered_asso.to_csv("/home/share/aschoi/data/Drug_Repositioning/7_new_association/5_filtered_association(drug).tsv", sep="\t", index=False)
filtered_asso.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/5_filtered_association(drug).tsv", sep="\t", index=False)

In [24]:
df_sider = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3_3_SIDER_STITCHid_DrugBankid.tsv")
df_offside = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/4_3_OFFSIDE_STITCHid_DrugBankid.tsv")

In [25]:
print(len(df_sider))
df_sider.head()

801


,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID
0,CID100002656,CID005479530,DB01212
1,CID100002650,CID005481173,DB00438
2,CID100002771,CID000002771,DB00215
3,CID100002617,CID000033255,DB01327
4,CID100002764,CID000002764,DB00537


In [26]:
print(len(df_offside))
df_offside.head()

558


,STITCH_compound_id_stereo,DrugBank_ID
0,CID000000119,DB02530
1,CID000000338,DB00936
2,CID000000444,DB01156
3,CID000000564,DB00513
4,CID000000596,DB05494


In [27]:
sider_drug = dr[dr['drug_id'].isin(df_sider['DrugBank_ID'])]
print len(sider_drug)
offside_drug = dr[dr['drug_id'].isin(df_offside['DrugBank_ID'])]
print len(offside_drug)

404
303


In [28]:
a = pd.concat([df_sider['DrugBank_ID'], df_offside['DrugBank_ID']]) # sider의 drug와 offside의 drug 합친거
b = a.drop_duplicates() # 중복제거

In [29]:
print len(df_sider['DrugBank_ID'].drop_duplicates())
print len(df_offside['DrugBank_ID'].drop_duplicates())
print len(a)
print len(b)

801
558
1359
1034


In [30]:
print len(dr)
print len(dr[dr['drug_id'].isin(b)]) # 제공되는 dataset중 feature가 존재하는 drug 수

663
500


1.0

2532


,disease_id,drug_id
0,D102100,DB00555


In [19]:
print len(pair[pair['drug_id'].isin(df_sider['DrugBank_ID'])].drop_duplicates(subset='drug_id'))
pair[pair['drug_id'].isin(df_sider['DrugBank_ID'])].drop_duplicates(subset='drug_id').head()

404


,disease_id,drug_id
0,D102100,DB00555
0,D102100,DB00949
0,D102100,DB01068
0,D102300,DB00190
0,D102300,DB00193


In [20]:
print len(pair[pair['drug_id'].isin(df_offside['DrugBank_ID'])].drop_duplicates(subset='drug_id'))
pair[pair['drug_id'].isin(df_offside['DrugBank_ID'])].drop_duplicates(subset='drug_id').head()

303


,disease_id,drug_id
0,D102100,DB00555
0,D102100,DB00949
0,D102100,DB01068
0,D102300,DB00268
0,D102300,DB00564


In [21]:
print len(pair['drug_id'].drop_duplicates())
print len(pair[pair['drug_id'].isin(b)].drop_duplicates(subset='drug_id'))
print len(pair)
print len(pair[pair['drug_id'].isin(b)])
pair[pair['drug_id'].isin(b)].drop_duplicates(subset='drug_id').head()

663
500
2532
1947


,disease_id,drug_id
0,D102100,DB00555
0,D102100,DB00949
0,D102100,DB01068
0,D102300,DB00190
0,D102300,DB00193


In [23]:
c = pair[pair['drug_id'].isin(b)] # cdataset의 association중, feature가 존재하는 drug 찾은것.
print len(c)
print(len(pair.drop_duplicates(subset='disease_id')))
print(len(c.drop_duplicates(subset='disease_id')))

1947
409
364


In [25]:
print len(b)

1034


In [26]:
print len(c)
c.head(1)

1947


,disease_id,drug_id
0,D102100,DB00555


In [23]:
len(c['drug_id'].drop_duplicates())

500

In [24]:
print len(pair['disease_id'].drop_duplicates())
print len(disease_id)

409
409


In [27]:
c.to_csv("/home/share/aschoi/data/Drug_Repositioning/7_new_association/5_before_gold_drug.tsv", sep='\t', index=False)

In [28]:
c.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/5_before_gold_drug.tsv", sep='\t', index=False)